## Remote ID Attack

In [1]:
import numpy as np

# Custom modules
from helpers import global2local, plot_3d_interactive, kill_processes
from simulators import Simulator, Gazebo, QGC,Color
from oracle import Oracle

from plan import Plan, State
from vehicle_logic import VehicleMode

kill_processes()

## Create Plans

In [2]:
offsets = [(-10, 0, 0, 0),(0, -5, 0, 0)]#
n_vehicles=len(offsets)

homes=np.array([offset[:3] for offset in offsets])


radar_radius=10
safety_radius =5


## Normal behavior
# global_paths=[np.array([[-10, 0, 5],[10, 0, 5]]),
#               np.array([[0, -5, 5]])]
# local_paths=[global2local(path, home,pairwise=True) for path,home in zip(global_paths,homes)]
# markers = {f'waypoints': {'pos':global_paths[0],'color':'green'},
#        f'real pos': {'pos':global_paths[1],'color':'red','radius':0.1},
#        f'avoid zone': {'pos':global_paths[1],'color':'orange','radius':safety_radius,'alpha': 0.84},
#        f'radar zone': {'pos':global_paths[1],'color':'orange','radius':radar_radius,'alpha': 0.9}}


## Remote ID attack
global_paths=[np.array([[-10, 0, 5],[10, 0, 5]]),
              np.array([[0, -5, 5]])]
local_paths=[global2local(path, home) for path,home in zip(global_paths,homes)]
fake_posistion=np.array([[0,0,5]])
markers = {f'waypoints': {'pos':global_paths[0],'color':Color.GREEN},
       f'real pos': {'pos':global_paths[1],'color':Color.RED,'radius':0.1},
       f'fake_pos': {'pos':fake_posistion,'color':Color.ORANGE,'radius':0.1},
       f'avoid zone': {'pos':fake_posistion,'color':Color.RED,'radius':safety_radius,'alpha': 0.85},
       f'radar zone': {'pos':global_paths[1],'color':Color.ORANGE,'radius':radar_radius,'alpha': 0.9}
       }


plans=[Plan.basic(wps=local_paths[0],wp_margin=1,navegation_speed=1.5),
       Plan.hover(wps=local_paths[1],wp_margin=1,navegation_speed=5)
       ]

## Visualization Parameters

In [3]:
markers_plot = {f'waypoints': {'pos':global_paths[0],'color':Color.GREEN},
                f'uav start': {'pos':np.array([[-10, 0, 0]]),'color':Color.GREEN},
                'uav goal': {'pos':np.array([[10, 0, 0]]),'color':Color.GREEN},
              f'adversary pos': {'pos':global_paths[1],'color':Color.RED,'radius':0.1},
              #f'adversary fake pos': {'pos':fake_posistion,'color':'orange','radius':0.1}
              }
plot_3d_interactive(markers_plot,title='Simulation Markers',expand=[0.4,0.4,0.2],ground=-0.05)

## Choose Simulator

In [4]:
# simulator=QGC(offsets=offsets,
#               plans=plans,
#               origin=(-35.3633245,149.1652241,0,0))

simulator=Gazebo(offsets=offsets,
                 plans=plans,
                 world_path='simulators/gazebo/worlds/runway.world',
                 models=n_vehicles*['iris'],
                 colors = [Color.GREEN,Color.RED],
                 markers=markers)

Launch Simulator

In [5]:
uavs = simulator.launch()
for uav in uavs:
    uav.safety_radius = safety_radius
    uav.radar_radius = radar_radius
orac = Oracle(uavs)

Vehicle 1 launched 🚀
Vehicle 2 launched 🚀


## Execute Plan

In [6]:
while orac.vehs:
    orac.gather_broadcasts()
    ## Remote ID attack
    orac.pos[uavs[1]]=fake_posistion[0]
    for uav in orac.vehs.copy():
        orac.update_neighbors(uav)
        if uav.plan.state != State.DONE:
            uav.act()
        elif uav.idx==1 or len(orac.vehs)==1:
            orac.remove(uav)
kill_processes()

Vehicle 1: ▶️ Plan Started: 📋 basic
Vehicle 1: ▶️ Action Started: 🔧 PREARM
Vehicle 1: ▶️ Step Started: Check disarmed
Vehicle 2: ▶️ Plan Started: 📋 hover
Vehicle 2: ▶️ Action Started: 🔧 PREARM
Vehicle 2: ▶️ Step Started: Check disarmed
Vehicle 2: ✅ Step Done: Check disarmed
Vehicle 2: ▶️ Step Started: Check EKF
Vehicle 1: ✅ Step Done: Check disarmed
Vehicle 1: ▶️ Step Started: Check EKF
Vehicle 1: ✅ Step Done: Check EKF
Vehicle 1: ▶️ Step Started: Check GPS
Vehicle 2: ✅ Step Done: Check EKF
Vehicle 2: ▶️ Step Started: Check GPS
Vehicle 1: ✅ Step Done: Check GPS
Vehicle 1: ▶️ Step Started: Check system
Vehicle 2: ✅ Step Done: Check GPS
Vehicle 2: ▶️ Step Started: Check system
Vehicle 1: ✅ Step Done: Check system
Vehicle 1: ✅ Action Done: 🔧 PREARM
Vehicle 1: ▶️ Action Started: ⚙️ MODE: GUIDED
Vehicle 1: ▶️ Step Started: Switch to GUIDED
Vehicle 1: ✅ Step Done: Switch to GUIDED
Vehicle 1: ✅ Action Done: ⚙️ MODE: GUIDED
Vehicle 1: ▶️ Action Started: 🎚️ CHANGE_NAV_SPEED
Vehicle 1: ▶️ Step S

In [7]:
uavs[0].safety_radius

IndexError: list index out of range